In [1]:
"|IMPORT PACKAGES|"
import numpy              as np
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Text, Span
from   bokeh.models.tools import HoverTool
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   bokeh.transform    import dodge
from   datetime           import datetime as dt

In [56]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'ANUAL'

data = pd.read_excel(IO, sheet_name = sheet, usecols="B,G,J:N", skiprows=2, nrows=28, engine='openpyxl') # Be patient...

data = data.rename(columns={"Var. Reservas"                       :"TOTAL"})
data = data.rename(columns={"Compra de Divisas"                   :"DIVISAS"})
data = data.rename(columns={"Organismos Internacionales"          :"INTL"})
data = data.rename(columns={"Otras Operaciones del Sector Publico":"PUBLICO"})
data = data.rename(columns={"Efectivo Minimo"                     :"EFECTIVO"})
data = data.rename(columns={"Otros"                               :"OTROS"}) 

data['TOTAL']    = data['TOTAL']   /1000
data['DIVISAS']  = data['DIVISAS'] /1000
data['INTL']     = data['INTL']    /1000
data['PUBLICO']  = data['PUBLICO'] /1000
data['EFECTIVO'] = data['EFECTIVO']/1000
data['OTROS']    = data['OTROS']   /1000

data['YEAR2'] = data['YEAR'].dt.year

In [57]:
"|CHECK DATA|"
data

,YEAR,TOTAL,DIVISAS,INTL,PUBLICO,EFECTIVO,OTROS,YEAR2
0,1993-12-31,NaN,NaN,NaN,NaN,NaN,NaN,1993
1,1994-12-31,NaN,NaN,NaN,NaN,NaN,NaN,1994
2,1995-12-31,NaN,NaN,NaN,NaN,NaN,NaN,1995
3,1996-12-31,NaN,NaN,NaN,NaN,NaN,NaN,1996
4,1997-12-31,NaN,NaN,NaN,NaN,NaN,NaN,1997
5,1998-12-31,NaN,NaN,NaN,NaN,NaN,NaN,1998
6,1999-12-31,-624.432421,NaN,NaN,NaN,NaN,NaN,1999
7,2000-12-31,-375.552315,NaN,NaN,NaN,NaN,NaN,2000
8,2001-12-31,1017.179736,NaN,NaN,NaN,NaN,NaN,2001
9,2002-12-31,-9316.222173,NaN,NaN,NaN,NaN,NaN,2002


In [58]:
"|BUILD PLOT|"

cds = ColumnDataSource(data)
colors = ['green', 'red', 'gray','blue','yellow']

#BUILD FIGURE
p = figure(title = "RESERVAS DEL BCRA",
#           y_range      = (-40000, 40000),
           x_range      = (2002,2021),
           x_axis_label = "",
           y_axis_label = "Miles de millones de dólares",
           plot_height  = 400,
           plot_width   = 700)

p.toolbar_location = "above"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p1 = p.vbar(x=dodge('YEAR2', -0.30, range=p.x_range), top='DIVISAS' , width=0.15, fill_color='green' , fill_alpha=0.75, source=cds)
p2 = p.vbar(x=dodge('YEAR2', -0.15, range=p.x_range), top='INTL'    , width=0.15, fill_color='red'   , fill_alpha=0.75, source=cds)
p3 = p.vbar(x=dodge('YEAR2',  0.00, range=p.x_range), top='PUBLICO' , width=0.15, fill_color='gray'  , fill_alpha=0.75, source=cds)
p4 = p.vbar(x=dodge('YEAR2',  0.15, range=p.x_range), top='EFECTIVO', width=0.15, fill_color='blue'  , fill_alpha=0.75, source=cds)
p5 = p.vbar(x=dodge('YEAR2',  0.30, range=p.x_range), top='OTROS'   , width=0.15, fill_color='yellow', fill_alpha=0.75, source=cds)




#p.vbar(x=[1, 2, 3], width=0.5, bottom=0,
       #top=[1.2, 2.5, 3.7], color="firebrick")

#p.vbar(bottom=stack()         , top=stack('DIVISAS')      , x='YEAR', color='green', source=cds, name='Compra de divisas')
#p.vbar(bottom=stack('DIVISAS'), top=stack('DIVISAS','INT'), x='YEAR', color='red'  , source=cds, name="Organismos internacionales")

#p1 = p.line('WEEK', 'RESERVAS', color='black', line_alpha=0.75, width=2, legend_label='Reservas brutas'  , muted_alpha=0.2, source=cds)


#p.add_tools(HoverTool(renderers=[p1], tooltips = [("Reservas Brutas"  , "@RESERVAS")], mode="vline"))

#LEGEND
#h_axis = Span(location=0, dimension="width", line_color="black")
#p.add_layout(h_axis)
#p.xaxis[0].formatter = DatetimeTickFormatter(months='%b %Y')
p.yaxis[0].formatter = NumeralTickFormatter(format="$0")
#p.legend.location     = "top_left"
#p.legend.orientation  = "vertical"
#p.legend.click_policy = "mute"
show(p)